## Scraping antara news

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import threading

In [2]:
jumlah_index = 1
threads_link = []
links = []
results = []
threads = []

In [3]:
def scrape_links(page_number):
    url = f"https://www.antaranews.com/olahraga/{page_number}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('article',{"class": "simple-post simple-big clearfix"})
    
    page_links = []
    for article in articles:
        link = article.find('a')['href']
        page_links.append(link)
    
    print(f"Scraped {len(page_links)} links from page {page_number}")
    return page_links

In [4]:
for page_number in range(1, jumlah_index + 1):
    thread = threading.Thread(target=lambda p=page_number: links.extend(scrape_links(p)))
    thread.start()
    threads_link.append(thread)

for thread in threads_link:
    thread.join()
print("Total Links:", len(links))

KeyboardInterrupt: 

In [ ]:
def scrape_url(url):
    try:
        headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
        }
        response = requests.get(url,headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')        
            # Judul Berita
            title_elem = soup.find('h1',{"class": "post-title"})
            if title_elem:
                title_text = title_elem.text
            else:
                title_text = "Title not found"
            # Author berita
            author_elem = soup.find('div', {"class": "tags-wrapper"})
            if author_elem:
                author_text = author_elem.find('span')
                author_text = author_text.text.split(':')[1]
            else:
                author_text = "Author not found"     
            # tanggal berita
            date_elem = soup.find('span', {"class": "article-date"})
            if date_elem:
                date_text = date_elem.text.strip()
            else:
                date_text = "Date not found"
                # Category berita
            category_elements = soup.find('ul', {"class": "breadcrumb"})
            if category_elements:
                category_text = category_elements.find_all('li')
                category_text= category_text[2].get_text()
            else:
                category_text = "Category not found"
            #     # Content Berita
            body_elem = soup.find('div', {"class": "post-content clearfix"})
            if body_elem:
                 content_text = body_elem.text
            else:
                content_text ="Content not found"

            results.append({'title': title_text,
                            'author' : author_text,
                            'category':category_text,
                            'date': date_text,
                            'content' : content_text,
                            'link' : url})
        else:
            print(f"Failed to retrieve data from {url}")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL '{url}': {e}")
    except Exception as e:
        print(f"Error processing URL '{url}': {e}")

In [ ]:
for url in links:
    thread = threading.Thread(target=scrape_url, args=(url,))
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

Error processing URL 'https://www.antaranews.com/berita/3788034/garuda-dan-baladewa-kunci-gelar-liga-sofbol-indonesia-2023': 'NoneType' object has no attribute 'text'
Error processing URL 'https://www.antaranews.com/berita/3788640/sebanyak-79-tim-akan-bersaing-dalam-kejurnas-voli-pantai-2023': 'NoneType' object has no attribute 'text'
Error processing URL 'https://www.antaranews.com/berita/3788613/pelatih-nilai-bagas-fikri-belum-tampil-optimal-pada-final-denmark-open': 'NoneType' object has no attribute 'text'
Error processing URL 'https://www.antaranews.com/berita/3788265/avila-bahar-optimistis-raih-hasil-memuaskan-di-seri-pamungkas-issom-2023': 'NoneType' object has no attribute 'text'
Error processing URL 'https://www.antaranews.com/berita/3788646/pb-wi-berharap-akan-ada-gelaran-imag-edisi-kedua-untuk-bina-atlet': 'NoneType' object has no attribute 'text'
Error processing URL 'https://www.antaranews.com/berita/3788409/ketua-panpel-bni-indonesia-masters-2023-persaingannya-lebih-sengi

In [ ]:
df = pd.DataFrame(results)
print("hasil data scrapping",len(results))
df.head(100)

hasil data scrapping 3


,title,author,category,date,content,link
0,Pelari Saptoyogo sabet medali emas pertama unt...,Author not found,Category not found,"Selasa, 24 Oktober 2023 07:58 WIB",Content not found,https://www.antaranews.com/foto/3788712/pelari...
1,Maulana Rifky Yavianda raih emas Asian Para Games,Author not found,Category not found,"Senin, 23 Oktober 2023 23:01 WIB",Content not found,https://www.antaranews.com/foto/3788598/maulan...
2,Pelari Saptoyogo sumbang emas pertama Asian Pa...,Author not found,Category not found,"Senin, 23 Oktober 2023 21:23 WIB",\n\n \n\n\n\n\n\n\n\n\n\n\t\t\t\t\t\t\t\tANTAR...,https://www.antaranews.com/video/3788427/pelar...


Tempat Simpan Save

In [ ]:
# current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
# excel_file_name = f'../tempat_simpan_nasional/antaranews_{current_datetime}.xlsx'
# df.to_excel(excel_file_name, index=False)

# print(f'Data has been saved to {excel_file_name}')